# Creating the HTLC-Timeout Transaction

In this section, we'll build a Lightning channel HTLC-timeout transaction from scratch using Python. We'll walk through each part of the transaction — how it's constructed and signed. The process will be tested using Bitcoin Core in regtest mode.

## Setup

For this notebook, we'll use the commitment transaction created in `chapter 3 - in-flight htlc commitment transaction` which contains the offered HTLC output that we'll spend with the HTLC-timeout transaction.

In [1]:
%run "../chapter 3 - in-flight htlc commitment transaction/in-flight htlc commitment transaction.ipynb"

2026-02-04T02:44:40.889000Z TestFramework (INFO): PRNG seed is: 8898014893640350402
2026-02-04T02:44:40.891000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_51lzv04l
🟢 New TestShell started. Block height: 0
Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Payment BasePoint:  025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Bob Payment BasePoint:  02f98efd3f2b2fbe7bd83c419f5f64f8280798b8a9175fdb77c0091bbb95c79506
To obscure commitment number 0xb433fd43a66f
Alice funding pubkey: 9310d6a4713b6f1da4a8c52ff577ce3e2f3efe5b72c820cfa89fc041bd3487b6
Alice funding privkey: 1869db5f24aa3990f9511a6265ca2c859a1175477bffd64ce08cbc9aebd4585c
Alice funding address: bcrt1pjvgddfr38dh3mf9gc5hl2a7w8chnaljmwtyzpnagnlqyr0f5s7mqnllca2
Bob funding pubkey: be7934c9c8193372e69fbdf44dd68b354a5a704fb4930c20592d7024567a0e2f
Bob fundi

## HTLC-Timeout Transaction

The HTLC-timeout transaction is a second-level transaction that spends an offered HTLC output from a commitment transaction. It allows the local party (Alice, who offered the HTLC) to reclaim the funds after the timeout period if the remote party (Bob) doesn't claim the HTLC with the preimage.

### The Unsigned HTLC-Timeout Transaction

#### The Input

The input is the offered HTLC output from Alice's commitment transaction (output index 3).

In [2]:
# Get the commitment transaction details
# Alice's second commitment transaction has the offered HTLC output at index 3
commitment_txid = decoded_alice_commitment_tx['txid']
commitment_output_index = 3

print(f"Spending from commitment tx: {commitment_txid}")

# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
commitment_txid = bytes.fromhex(commitment_txid)[::-1]
commitment_index = commitment_output_index.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# Sequence: must be set to enforce relative locktime (though scriptA doesn't have CSV)
# We use sequence 0xfffffffe to enable RBF and relative locktimes
sequence_htlc = bytes.fromhex("feffffff")

inputs = (
    commitment_txid
    + commitment_index
    + varint_len(scriptsig)
    + scriptsig
    + sequence_htlc
)

Spending from commitment tx: 5a179a03c790073ad2fee9262ef1f760bc946d4e2ac07e828912b7f0de095409


#### The Output

The HTLC-timeout transaction has a single output that pays to Alice with a delay. This output is similar to the `to_local` output in commitment transactions: it's spendable by Alice after a delay, or immediately by Bob if he has the revocation key (in case this commitment gets revoked).

#### To Local Delayed Output

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +-------------------+
                    ---| P(revocation) + T |
                       +-------------------+
                                         ^
                                         |
                                   +-----------+        
                                   | T = t * G |
                                   +-----------+        
                                         ^
                                         |
     +---+   +-------------------------------------------------------+
     | t | = | TaggedHash ("Taptweak", P(revocation) || script_root) |
     +---+   +-------------------------------------------------------+
                                                             ^
                                                             |
                                                          +-----+
                                                          |  h  |
                                                          +-----+                   
                                                             ^                                                          
                                                             |                                                          
      +-----------------------------------------------------------+                                           
      | P(local_delayed) OP_CHECKSIG to_self_delay OP_CSV OP_DROP |                                           
      +-----------------------------------------------------------+

Obs: If option_anchors applies, which is the case here, then the HTLC-timeout and HTLC-success transactions are signed with the input and output having the same value. This means they have a zero fee and MUST be combined with other inputs to arrive at a reasonable fee.

In [3]:
# Output count
output_count = bytes.fromhex("01")

# Output value
output_value_sat = htlc_output_value_satoshis
output_value = output_value_sat.to_bytes(8, byteorder="little", signed=False)

# We already have alice_delayed_pubkey and bob_revocation_pubkey from chapter 3
# These are the same keys used in the commitment transaction to_local output

# Create the script: P(local_delayed) OP_CHECKSIG to_self_delay OP_CSV OP_DROP
script_output = CScript([alice_delayed_pubkey.get_bytes(), OP_CHECKSIG, to_self_delay, OP_CHECKSEQUENCEVERIFY, OP_DROP])

# Compute output script_root
hash_input = TAPSCRIPT_VER + ser_string(script_output)
script_root = tagged_hash("TapBranch", hash_input)

# Compute the output Tagged Hash
taptweak = tagged_hash("TapTweak", bob_revocation_pubkey.get_bytes() + script_root)
htlc_bob_revocation_pubkey_tweaked = bob_revocation_pubkey.tweak_add(taptweak)
# Compute scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
output_spk = bytes.fromhex("51") + varint_len(htlc_bob_revocation_pubkey_tweaked.get_bytes()) + htlc_bob_revocation_pubkey_tweaked.get_bytes()

outputs = (
    output_value
    + varint_len(output_spk)
    + output_spk
)

# Locktime: set to current block height + time lock delta
current_block_height = node.getblockcount()
locktime = current_block_height + 60  # assuming a 60-block of time lock delta
locktime = locktime.to_bytes(4, 'little')

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("\nunsigned_tx:", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded_htlc_timeout = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded_htlc_timeout, indent=2, default=str))


unsigned_tx: 0200000001095409def0b71289827ec02a4e6d94bc60f7f12e26e9fed23a0790c7039a175a0300000000feffffff0120a10700000000002251209c4e1af0245ce5eace1cc1432fe0d0140ca6de1e289aa4e5d34c19d4b7efafdfa3000000
{
  "txid": "7de050309614e177046717404cf216d743cafbe5d70565bec6656e270faa817b",
  "hash": "7de050309614e177046717404cf216d743cafbe5d70565bec6656e270faa817b",
  "version": 2,
  "size": 94,
  "vsize": 94,
  "weight": 376,
  "locktime": 163,
  "vin": [
    {
      "txid": "5a179a03c790073ad2fee9262ef1f760bc946d4e2ac07e828912b7f0de095409",
      "vout": 3,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967294
    }
  ],
  "vout": [
    {
      "value": "0.00500000",
      "n": 0,
      "scriptPubKey": {
        "asm": "1 9c4e1af0245ce5eace1cc1432fe0d0140ca6de1e289aa4e5d34c19d4b7efafdf",
        "desc": "rawtr(9c4e1af0245ce5eace1cc1432fe0d0140ca6de1e289aa4e5d34c19d4b7efafdf)#c0d4ugzn",
        "hex": "51209c4e1af0245ce5eace1cc1432fe0d0140ca6de1e289aa

## The sighash for script path spend

Unlike the commitment transaction which uses key path spending, the HTLC-timeout transaction uses **script path spending** through the Taproot tree. We need to spend using scriptA of the offered HTLC output.

The sighash calculation for script path spending is similar to key path, but includes additional data about the script being executed.

In [4]:
# We're spending the offered HTLC output using scriptA
# Recall scriptA from chapter 3: P(local_htlc) OP_CHECKSIGVERIFY P(remote_htlc) OP_CHECKSIG
spending_script = CScript([alice_htlc_pubkey.get_bytes(bip340=True), OP_CHECKSIGVERIFY, bob_htlc_pubkey.get_bytes(bip340=True), OP_CHECKSIG])

print("\n bob_htlc_pubkey.get_bytes(bip340=True):", bob_htlc_pubkey.get_bytes(bip340=True).hex())  # Debug line
print("\n alice_htlc_pubkey.get_bytes(bip340=True):", alice_htlc_pubkey.get_bytes(bip340=True).hex())  # Debug line

# Calculate the tapleaf hash for the script we're spending
hash_input_spending = TAPSCRIPT_VER + ser_string(spending_script)
tapleaf_hash = tagged_hash("TapLeaf", hash_input_spending)
key_version = bytes.fromhex("00")  # reserved for future ugrades
codeseparator = bytes.fromhex("ffffffff")  

print("\ntapleaf_hash:", tapleaf_hash.hex())

# SIGHASH for script path spend (BIP-341)
index_of_this_input = bytes.fromhex("0000 0000")
sighash_epoch = bytes.fromhex("00")
hash_type = bytes.fromhex("83")  # SIGHASH_SINGLE|SIGHASH_ANYONECANPAY (0x03 | 0x80)

# When SIGHASH_SINGLE (0x03) is set, these fields are 35 bytes (BIP-341)
# corresponding to the output (32): the SHA256 of the corresponding output in CTxOut format.
sha_outputs = sha256(outputs).digest()

# Data about this input (script path spend)
spend_type = bytes.fromhex("02")  # script path (no annex)

# Common signature message extension (BIP-341)
# When SIGHASH_ANYONECANPAY is set, we include data about THIS specific input:
outpoint = commitment_txid + commitment_index  # 36 bytes
amount = htlc_output_value  # 8 bytes
nSequence = sequence_htlc  # 4 bytes

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + spend_type
    + outpoint          
    + amount
    + varint_len(alice_offered_htlc_spk)
    + alice_offered_htlc_spk
    + nSequence
    + sha_outputs
    + tapleaf_hash
    + key_version
    + codeseparator
)

print("\nsig_msg:", sig_msg.hex())

tag_hash = sha256("TapSighash".encode()).digest()
sighash = sha256(tag_hash + tag_hash + sig_msg).digest()
print("\nSighash:", sighash.hex())


 bob_htlc_pubkey.get_bytes(bip340=True): 7bd8383df06640c0efe760bb2bac1a1585bdc664e6f1cb61c41a76a6f2be3484

 alice_htlc_pubkey.get_bytes(bip340=True): 7546c6ab3a8c005ce24e9feb7154fd35b62ac0f9125beb5e692d4d912a95de77

tapleaf_hash: 509fbe809f2233d8daa3bca96c2735a7f7a1bd75eb48976978029ac95cd0fd13

sig_msg: 008302000000a300000002095409def0b71289827ec02a4e6d94bc60f7f12e26e9fed23a0790c7039a175a0300000020a1070000000000225120ba9dfd92bdd3bb7b3b4faa510e1076c32c5105c0f0e620d238486e8fa8467383feffffff37b1a6d47e215cb28259eac50b4162048f0c004a4b0fde33a8532cd25b5a1f1b509fbe809f2233d8daa3bca96c2735a7f7a1bd75eb48976978029ac95cd0fd1300ffffffff

Sighash: 7fd22703cbb7c95bd0206fab3a2ea97aee7490f425059f79f6ba72f7e0346e11


## Signing the sighash

For the HTLC-timeout transaction, we need signatures from both Alice's and Bob's HTLC keys. These are regular Schnorr signatures (not MuSig2 aggregated signatures), since the script requires two separate signature checks.

In [5]:
# Derive Alice's HTLC private key
alice_htlc_basepoint = derivate_key(alice_node_seed, family=2, channel_index=0)
alice_htlc_privkey = alice_htlc_basepoint.get_privkey(alice_per_commitment.get_pub())

# Derive Bob's HTLC private key
bob_htlc_basepoint = derivate_key(bob_node_seed, family=2, channel_index=0)
bob_htlc_privkey = bob_htlc_basepoint.get_privkey(bob_per_commitment.get_pub())

# Generate auxiliary random data for each signature
import secrets
aux_alice = secrets.token_bytes(32)
aux_bob = secrets.token_bytes(32)

# Sign with Alice's HTLC key
alice_htlc_sig = alice_htlc_privkey.sign_schnorr(sighash, aux_alice)
# APPEND sighash type byte for non-default sighash!
alice_htlc_sig = alice_htlc_sig + hash_type  # Add 0x83 byte

# Verify Alice's signature (without the sighash byte for verification)
alice_sig_valid = alice_htlc_pubkey.verify_schnorr(alice_htlc_sig[:-1], sighash)
print("Alice signature valid?", alice_sig_valid)
print(f"Alice signature length: {len(alice_htlc_sig)} bytes (should be 65)")

# Sign with Bob's HTLC key  
bob_htlc_sig = bob_htlc_privkey.sign_schnorr(sighash, aux_bob)
# APPEND sighash type byte for non-default sighash!
bob_htlc_sig = bob_htlc_sig + hash_type  # Add 0x83 byte

# Verify Bob's signature (without the sighash byte for verification)
bob_sig_valid = bob_htlc_pubkey.verify_schnorr(bob_htlc_sig[:-1], sighash)
print("Bob signature valid?", bob_sig_valid)
print(f"Bob signature length: {len(bob_htlc_sig)} bytes (should be 65)")

Alice signature valid? True
Alice signature length: 65 bytes (should be 65)
Bob signature valid? True
Bob signature length: 65 bytes (should be 65)


## The signed transaction

Now we construct the witness for the script path spend. The witness stack for spending scriptA contains:
1. Bob's signature (last signature on stack, for OP_CHECKSIG)
2. Alice's signature (for OP_CHECKSIGVERIFY)
3. The script being executed (scriptA)
4. The control block (proving scriptA is in the taproot tree)

In [6]:
# Construct the control block
# Control block format: <version byte> <internal key> <parity bit> [<merkle proof>]
# Version byte: 0xc0 | (parity of Q)

# Get parity bit from Q
parity = 0 if bob_revocation_pubkey_tweaked.get_bytes(bip340=True)[0] == 0x02 else 1
version_byte = bytes([0xc0 | parity])

control_block = version_byte + bob_revocation_pubkey.get_bytes(bip340=True) + htlc_taggedhash_leafB

print("\ncontrol_block:", control_block.hex())
print("\nspending_script:", spending_script.hex())

# Construct witness stack
# Stack order (bottom to top): <alice_sig> <bob_sig> <script> <control_block>
witness = (
    bytes.fromhex("04")  # 4 stack items
    + varint_len(bob_htlc_sig)    # Bob's signature (for OP_CHECKSIG)
    + bob_htlc_sig
    + varint_len(alice_htlc_sig)  # Alice's signature (for OP_CHECKSIGVERIFY)
    + alice_htlc_sig
    + varint_len(spending_script)  # The script
    + spending_script
    + varint_len(control_block)    # The control block
    + control_block
)

# The final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("\nsigned tx:", signed_tx.hex())

# Decode the signed transaction
decoded_signed = node.decoderawtransaction(signed_tx.hex())
print("\n" + json.dumps(decoded_signed, indent=2, default=str))

# Test if it would be accepted (note: we can't actually broadcast it without confirm the commitment tx first)
print("\nTest mempool accept:")
try:
    result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
    print(json.dumps(result, indent=2, default=str))
except Exception as e:
    print(f"Expected error (commitment tx not in mempool): {e}")


control_block: c1c39a6b0cffa569f243cec8252a1e5f93b4b072247ff18773917b72fb6341f6088da58f2aee0bccdd60c6065603a5d2ab655e79971b068228afbccbb02220f54b

spending_script: 207546c6ab3a8c005ce24e9feb7154fd35b62ac0f9125beb5e692d4d912a95de77ad207bd8383df06640c0efe760bb2bac1a1585bdc664e6f1cb61c41a76a6f2be3484ac

signed tx: 02000000000101095409def0b71289827ec02a4e6d94bc60f7f12e26e9fed23a0790c7039a175a0300000000feffffff0120a10700000000002251209c4e1af0245ce5eace1cc1432fe0d0140ca6de1e289aa4e5d34c19d4b7efafdf0441dead414828ff2e544fe9b82ce5d65075eb71601a79de400341d7b03302ac6b9c5664460784c62bdbdeb8529773225ab209417d5c52fd5566d22b34f80077486583412a2d3f9dcbb963ba25c322f308bddba3896aca941110e28d7c376b72849a0f1f351e87ad2415df7384433379716dd53394740b3b4ecc62a3b8a93dfc45489a1c8344207546c6ab3a8c005ce24e9feb7154fd35b62ac0f9125beb5e692d4d912a95de77ad207bd8383df06640c0efe760bb2bac1a1585bdc664e6f1cb61c41a76a6f2be3484ac41c1c39a6b0cffa569f243cec8252a1e5f93b4b072247ff18773917b72fb6341f6088da58f2aee0bccdd60c6065603a5d2

In [7]:
alice_commitment_txid = node.sendrawtransaction(signed_alice_commitment_tx.hex())
result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

print("alice_commitment_txid: ", alice_commitment_txid)

alice_commitment_txid:  5a179a03c790073ad2fee9262ef1f760bc946d4e2ac07e828912b7f0de095409


In [8]:
# We mine enough blocks to satisfy the htlc time lock delta but now we don´t fit the min relay fee.
# We need actually to add another input to pay the fee.
result = node.generatetoaddress(nblocks=59, address=address, called_by_framework=True)

print("\nTest mempool accept:")
try:
    result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
    print(json.dumps(result, indent=2, default=str))
except Exception as e:
    print(f"Expected error (commitment tx not in mempool): {e}")


Test mempool accept:
[
  {
    "txid": "7de050309614e177046717404cf216d743cafbe5d70565bec6656e270faa817b",
    "wtxid": "be003de3bb127a1c59aa20e198211fcf13e5583c492bcbdd94e0e12f01f04b89",
    "allowed": false,
    "reject-reason": "min relay fee not met",
    "reject-details": "min relay fee not met, 0 < 162"
  }
]


## Adding alice_change input for fees

As stated in the BOLTs: "If option_anchors applies, then the HTLC-timeout and HTLC-success transactions are signed with the input and output having the same value. This means they have a zero fee and MUST be combined with other inputs to arrive at a reasonable fee."

We'll now change the transaction adding alice_change output from the funding transaction as a second input to pay for fees.

In [9]:
# INPUTS
# We have 2 inputs: the HTLC output and alice_change from funding transaction for fees 
input_count = bytes.fromhex("02")


# INPUT 2: alice_change output from funding transaction for fees
# Get funding transaction details from chapter 1
alice_change_txid_bytes = bytes.fromhex(funding_channel_txid)[::-1]
alice_change_index = (1).to_bytes(4, byteorder="little", signed=False)
sequence_change = bytes.fromhex("ffffffff")

inputs = (
    commitment_txid
    + commitment_index
    + varint_len(scriptsig)
    + scriptsig
    + sequence_htlc
    + alice_change_txid_bytes
    + alice_change_index
    + varint_len(scriptsig)
    + scriptsig
    + sequence_change
)

print(f"Added alice_change input from funding tx: {funding_channel_txid}:1")

Added alice_change input from funding tx: 12b0f76a990c2d8d65bc816431b604bec6c139e6ff2f196925993ee99f5607c0:1


In [10]:
# OUTPUTS
# Now we have 2 outputs: HTLC delayed output + change back to alice
output_count_v2 = bytes.fromhex("02")

# OUTPUT 1: HTLC delayed output (same as before)
# Reuse the same output_value and output_spk from v1

# OUTPUT 2: Change back to alice_change address
# Calculate: alice_change_value (98,999,700) + htlc_value (500,000) - htlc_output (500,000) - fee
tx_fee_sat = 300
output_value_sat_change = alice_change_value_sat - tx_fee_sat
output_value_change = output_value_sat_change.to_bytes(8, byteorder="little", signed=False)

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + alice_change_pubkey
output_spk_change = bytes.fromhex("51") + varint_len(alice_change_pubkey.get_bytes()) + alice_change_pubkey.get_bytes()

# Use outputs_v2 to avoid overwriting the original outputs variable
outputs_v2 = (
    output_value
    + varint_len(output_spk)
    + output_spk
    + output_value_change
    + varint_len(output_spk_change)
    + output_spk_change
)

print(f"Output 1 (HTLC delayed): {output_value_sat} sats")
print(f"Output 2 (Change to Alice): {output_value_sat_change} sats")
print(f"Transaction fee: {tx_fee_sat} sats")

Output 1 (HTLC delayed): 500000 sats
Output 2 (Change to Alice): 98999400 sats
Transaction fee: 300 sats


In [11]:
unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count_v2
    + outputs_v2
    + locktime
)

print("unsigned_tx_v2:", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded_htlc_timeout = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded_htlc_timeout, indent=2, default=str))

unsigned_tx_v2: 0200000002095409def0b71289827ec02a4e6d94bc60f7f12e26e9fed23a0790c7039a175a0300000000feffffffc007569fe93e992569192fffe639c1c6be04b6316481bc658d2d0c996af7b0120100000000ffffffff0220a10700000000002251209c4e1af0245ce5eace1cc1432fe0d0140ca6de1e289aa4e5d34c19d4b7efafdf689ce60500000000225120d49b979ff7b6db12252d4a4c37b072a4a1dfe5bf7ef27421625933a5d82eca83a3000000
{
  "txid": "6d2993c4e7f0d692f2251cc461d15f6a4848e77cca535c3c117c2872e552eb58",
  "hash": "6d2993c4e7f0d692f2251cc461d15f6a4848e77cca535c3c117c2872e552eb58",
  "version": 2,
  "size": 178,
  "vsize": 178,
  "weight": 712,
  "locktime": 163,
  "vin": [
    {
      "txid": "5a179a03c790073ad2fee9262ef1f760bc946d4e2ac07e828912b7f0de095409",
      "vout": 3,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967294
    },
    {
      "txid": "12b0f76a990c2d8d65bc816431b604bec6c139e6ff2f196925993ee99f5607c0",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
    

### Sign the alice_change input (Input 1 - key path spend)

The HTLC input signatures were already created earlier and can be reused. We only need to sign the alice_change input using key path spending.

In [12]:
# Calculate sighash for alice_change input (input index 1)
index_of_change_input = bytes.fromhex("0100 0000")

# SIGHASH for key path spend
sighash_epoch = bytes.fromhex("00")
hash_type = bytes.fromhex("00")  # SIGHASH_DEFAULT (SIGHASH_ALL)

# Transaction data - now includes both inputs
sha_prevouts_v2 = sha256(commitment_txid + commitment_index + alice_change_txid_bytes + alice_change_index).digest()
sha_amounts_v2 = sha256(htlc_output_value + alice_change_value).digest()

# scriptPubKeys for both inputs
alice_offered_htlc_spk_with_len = varint_len(alice_offered_htlc_spk) + alice_offered_htlc_spk
alice_change_spk = bytes.fromhex("51") + varint_len(alice_change_pubkey.get_bytes()) + alice_change_pubkey.get_bytes()
alice_change_spk_with_len = varint_len(alice_change_spk) + alice_change_spk
sha_scriptpubkeys_v2 = sha256(alice_offered_htlc_spk_with_len + alice_change_spk_with_len).digest()

sha_sequences_v2 = sha256(sequence_htlc + sequence_change).digest()
sha_outputs_v2 = sha256(outputs_v2).digest()

# Data about this input (key path spend)
spend_type = bytes.fromhex("00")  # key path, no annex

sig_msg_change = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts_v2
    + sha_amounts_v2
    + sha_scriptpubkeys_v2
    + sha_sequences_v2
    + sha_outputs_v2
    + spend_type
    + index_of_change_input
)

print("\nsig_msg_change:", sig_msg_change.hex())

tag_hash = sha256("TapSighash".encode()).digest()
sighash_change = sha256(tag_hash + tag_hash + sig_msg_change).digest()
print("Sighash for alice_change input:", sighash_change.hex())

# Sign with alice_change_privkey
import secrets
aux_change = secrets.token_bytes(32)
alice_change_sig = alice_change_privkey.sign_schnorr(sighash_change, aux_change)

# Verify the signature
alice_change_sig_valid = alice_change_pubkey.verify_schnorr(alice_change_sig, sighash_change)
print("Alice change signature valid?", alice_change_sig_valid)


sig_msg_change: 000002000000a3000000f5a53e0a6a5965b5583c51e7f67ba482eeb49ede05869048117eddd35a0511f173c89fddfe2c01557ecb918a4ed29094917ab8df8630d8ff26098358801a27d18e033a2bea0c98cfff1fc04f494dc9463bbe4bed465b2ffd0ed6da4fd684907181b37f7f8b78762feb067131152a700548754f937a0668c1f327d2ee1a2d280149fbe9ed04aa54abf6c4e0ae3a91bd7d2a24bb7813e880ca1f0fe4f4fd77b7f20001000000
Sighash for alice_change input: 61fac74e1e8f9dbfd1c93916a0e9627c57a07dcf29fb4e79e7c0ada1f881c2ba
Alice change signature valid? True


### Build the final signed transaction

Now we construct the witness data for both inputs and build the final signed transaction.

In [13]:
# Witness for Input 1 (alice_change) - key path spend
witness_input1 = (
    bytes.fromhex("01")  # 1 stack item
    + varint_len(alice_change_sig)
    + alice_change_sig
)

# Complete witness data
witness_v2 = witness + witness_input1

# The final signed transaction
signed_tx_v2 = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count_v2
    + outputs_v2
    + witness_v2
    + locktime
)

print("signed tx v2:", signed_tx_v2.hex())

# Decode the signed transaction
decoded_signed_v2 = node.decoderawtransaction(signed_tx_v2.hex())
print("\n" + json.dumps(decoded_signed_v2, indent=2, default=str))

signed tx v2: 02000000000102095409def0b71289827ec02a4e6d94bc60f7f12e26e9fed23a0790c7039a175a0300000000feffffffc007569fe93e992569192fffe639c1c6be04b6316481bc658d2d0c996af7b0120100000000ffffffff0220a10700000000002251209c4e1af0245ce5eace1cc1432fe0d0140ca6de1e289aa4e5d34c19d4b7efafdf689ce60500000000225120d49b979ff7b6db12252d4a4c37b072a4a1dfe5bf7ef27421625933a5d82eca830441dead414828ff2e544fe9b82ce5d65075eb71601a79de400341d7b03302ac6b9c5664460784c62bdbdeb8529773225ab209417d5c52fd5566d22b34f80077486583412a2d3f9dcbb963ba25c322f308bddba3896aca941110e28d7c376b72849a0f1f351e87ad2415df7384433379716dd53394740b3b4ecc62a3b8a93dfc45489a1c8344207546c6ab3a8c005ce24e9feb7154fd35b62ac0f9125beb5e692d4d912a95de77ad207bd8383df06640c0efe760bb2bac1a1585bdc664e6f1cb61c41a76a6f2be3484ac41c1c39a6b0cffa569f243cec8252a1e5f93b4b072247ff18773917b72fb6341f6088da58f2aee0bccdd60c6065603a5d2ab655e79971b068228afbccbb02220f54b014061aa8690d2cbaabbb6d3e9f7d7e262094a1b4431eac451f0dd3c96162506dfb5670fe4f26ac8e55007753e97ec232d

In [14]:
# Test mempool accept
print("\nTest mempool accept:")
try:
    result = node.testmempoolaccept(rawtxs=[signed_tx_v2.hex()])
    print(json.dumps(result, indent=2, default=str))
    
    if result[0]['allowed']:
        print("\n✓ Transaction is valid and meets minimum relay fee!")
        print(f"  Fee: {result[0].get('fees', {}).get('base', 'N/A')} BTC")
    else:
        print(f"\n✗ Transaction rejected: {result[0].get('reject-reason', 'Unknown')}")
except Exception as e:
    print(f"Error: {e}")


Test mempool accept:
[
  {
    "txid": "6d2993c4e7f0d692f2251cc461d15f6a4848e77cca535c3c117c2872e552eb58",
    "wtxid": "d4dbdac1827fc13d295559b8adec18b49c99ef009bb532a32e9b11d85d37f41c",
    "allowed": true,
    "vsize": 262,
    "fees": {
      "base": "0.00000300",
      "effective-feerate": "0.00001145",
      "effective-includes": [
        "d4dbdac1827fc13d295559b8adec18b49c99ef009bb532a32e9b11d85d37f41c"
      ]
    }
  }
]

✓ Transaction is valid and meets minimum relay fee!
  Fee: 0.00000300 BTC


## Summary

In this chapter, we successfully created an HTLC-timeout transaction that spends the offered HTLC output from Alice's commitment transaction. Key points:

1. **Purpose**: The HTLC-timeout transaction allows Alice to reclaim funds from an offered HTLC after the timeout period if Bob doesn't provide the payment preimage.

2. **Script Path Spending**: Unlike the commitment transaction which uses key path spending, the HTLC-timeout uses script path spending through the Taproot tree.

3. **Two Signatures Required**: The timeout script (scriptA) requires signatures from both Alice's and Bob's HTLC keys, ensuring both parties agree to the timeout.

4. **Delayed Output**: The output of the HTLC-timeout transaction is similar to the `to_local` output in commitment transactions - it's spendable by Alice after a delay (to_self_delay), or immediately by Bob if he has the revocation key.

5. **Witness Structure**: The witness includes both signatures, the script being executed, and a control block proving the script is part of the Taproot tree.
